Tutorial Vanilla RNNs from the scratch (not really)

Alfan Farizki Wicaksono, Fasilkom UI

In [2]:
%pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.6 MB 2.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.6 MB 1.9 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.6 MB 2.0 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/11.6 MB 2.0 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.6 MB 2.0 MB/s eta 0:00:05
   --------- ------------------------------ 2.9/11.6 MB 2.1 MB/s eta 0:00:05
   ----------- ---------------------------- 3.4/11.6 MB 2.1 MB/s eta 0:00:04
   ------------ --------------------------- 3.7/11.6 MB 2.1 MB/s eta 0:00:04
   -------------- ------------------------- 4.2/11.6 MB 2.1 MB/s eta 0:00:04
   --------------- ------------------------ 4.5/11.6 MB 2.0 MB/s eta 0:00:04
   ----------------- ----


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


yang dilakukan di sini adalah:
- sebanyak max epoch dilakukan training (forward per batch). batch ada sebanyak daetaset // batch size
- setiap batch diforward. cara nge forward batch nya adalah dengan ngeforward setiap cell yang ada. Misal ada 6 cell berarti ada 6 kali forward cell untuk setiap batch.
- cara forward cell adalah dengan torch.tanh(self.h(hidden) + self.x(input))
- semua hasil forward-an cell disimpan di outputs dan hasil terakhir disimpan di hidden_state
- setiap selesai ngeproses per batch, dilakukan update parameter dengan optimizer.step()

In [36]:
import torch
import pandas as pd
import numpy as np

from collections import Counter
from torch import nn, optim
from torch.utils.data import DataLoader

In [37]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        sequence_length,
        documents, # list of strings
    ):
        self.sequence_length = sequence_length
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        text = ""
        for doc in documents:
          text += doc + " "
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

what model.train() does?


Sets Training Mode for Layers:

Dropout layers: In training mode, Dropout randomly zeroes out some neuron activations to prevent overfitting. When you call model.eval() (evaluation mode), dropout is disabled, and all neurons are used.
BatchNorm layers: Batch normalization layers calculate statistics (mean and variance) for each batch in training mode, updating internal running statistics to keep track of feature distribution. In evaluation mode, these running statistics are used instead of per-batch statistics, providing consistent behavior during inference.
Since your Model doesn’t include Dropout or BatchNorm, calling model.train() won’t have a direct impact on how your layers operate. However, if you add these types of layers in the future, calling model.train() would toggle them into training mode.

Ensures Consistency in Training Behavior:

Although model.train() doesn't directly alter parameters in models without Dropout or BatchNorm, it’s still a best practice to call it at the start of each training phase. This ensures that any layers that do depend on training mode are set up correctly, and it provides consistency across training and evaluation phases.

model bisa diset untuk train() atau untuk eval()

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = nn.Parameter(torch.Tensor(self.n_inputs, self.n_outputs))
        self.init_weights()

    def init_weights(self):
        for param in self.parameters():
            nn.init.uniform_(param, -0.1, 0.1)

    def forward(self, x):
        return x @ self.W

class RNNCell(nn.Module):
    def __init__(self, n_inputs, n_hiddens):
        super().__init__()
        self.h = Linear(n_hiddens, n_hiddens) # Whh
        self.x = Linear(n_inputs, n_hiddens) # Wxh

    def forward(self, input, hidden):
        print(f"ini hidden {hidden}")
        print(f"ini input {input}")
        return torch.tanh(self.h(hidden) + self.x(input))

class RNN(nn.Module):
    def __init__(self, n_inputs, n_hiddens, cell):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_hiddens = n_hiddens
        self.cell = cell(n_inputs, n_hiddens)

    def forward(self, inputs, prev_hidden_state):
        """ inputs: [batch_size, sequence_length, embedding_size] """ # embedding sama aja dgn channel kyk di CNN
        outputs = []
        print(f"all inputs {inputs}")
        hidden_state = prev_hidden_state
        n_steps = inputs.shape[1]
        for i in range(n_steps):
            print(f"ini inputs[:, i] {inputs[:, i]}")
            hidden_state = self.cell(inputs[:, i], hidden_state) # ini di RNNCell nya *semua sebelumnya* sampai i, trus hidden_state ini update h
            outputs.append(hidden_state) # ini output adalah hasil append semuanya tiap cell tanH(x*Wxh+h*Whh)
        

        return torch.stack(outputs, dim=1), hidden_state # output itu semuanya, state yg terakhir doang

class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.rnn_size = 16
        self.embedding_dim = 16
        self.n_vocab = len(dataset.uniq_words)
        # setiap kata di one hot encoding
        self.embedding = nn.Embedding(  # ini dari suatu kata yg di encode, harus di embed dulu supaya bbisa masuk ke RNN agar bentuknya jadi vector
            num_embeddings=self.n_vocab, # matrix encoding nya ukurannya sebanyak n_vocab*embedding_dim (karena yg dikali sebelumnya one hot encoded = sepanjang vocab)
            embedding_dim=self.embedding_dim # karena one hot encoding, maka setiap dikali adalah baris matrixnya
        ) # embedding matrix  ini di train bareng dgn RNN nya menggunakan grad descent kyk biasa
        self.rnn = RNN(
            n_inputs=self.embedding_dim,
            n_hiddens=self.rnn_size,
            cell=RNNCell
        )
        self.fc = Linear(self.rnn_size, self.n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x) # untuk setiap batch ke i, dijadiin embedding yg ukurannya (banyak_batch * banyak_x * embedding_dim)
        print(f"ini embedding {embed}. shapenya {embed.shape}")
        output, state = self.rnn(embed, prev_state)
        print(f"ini output {output}. shapenya {output.shape}")
        print(f"ini state {state}.shape nya {state.shape}")
        logits = self.fc(output)
        return logits, state

    def init_state(self, batch_size):
        return torch.zeros(batch_size, self.rnn_size)

def train(dataset, model, batch_size, max_epochs=2): # ganti balik max_epochs jadi 400
    model.train() # set to training mode

    dataloader = DataLoader(dataset, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(max_epochs):
        h_state = model.init_state(batch_size) #hidden state awal, NOL

        for batch, (x, y) in enumerate(dataloader): # si y ini ground truth (epected result dari x)
            y_pred, h_state = model(x, h_state) # ini forward
            loss = criterion(y_pred.transpose(1, 2), y) 

            loss.backward()
            optimizer.step() # disini di-update W nya
            # gimana cara updatenya?
            #in PyTorch, all learnable parameters are generally anything defined as nn.Parameter in the model. 
            # However, some nn.Module layers, like nn.Linear or nn.Embedding, automatically define their weights as nn.Parameter.

            
            #h_state detached from current graph; tapi isi tetap sama agar berlanjut (statefull)
            #graph batch sekarang jangan nyambung dengan batch berikutnya; tetapi nilai
            #h_state harus berlanjut dari satu batch ke batch berikutnya. Caranya adalah
            #dengan detach() ini.
            h_state = h_state.detach()

            optimizer.zero_grad()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() }) # dilakukan untuk setiap batch ke i

def predict(dataset, model, text, next_words=20):
    model.eval()

    words = text.split(' ')
    h_state = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, h_state = model(x, h_state)

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()

        # random choice
        #word_index = np.random.choice(len(last_word_logits), p=p)

        # the best one
        word_index = np.argmax(p)

        # selain best one bisa pake random choice atau yg lain, dan sebenarnya best choice bukan yg terbaik

        words.append(dataset.index_to_word[word_index])

    return words

In [34]:
# ini contoh pske dataset dan dataloader
import torch
from torch.utils.data import Dataset, DataLoader

# Custom Dataset class
class RandomDataset(Dataset):
    def __init__(self, length):
        self.length = length
        self.data = torch.randn(length, 10)  # 10 features per sample
        self.labels = torch.randint(0, 2, (length,))  # Binary labels (0 or 1)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create an instance of RandomDataset
dataset = RandomDataset(length=10)  # 100 samples in total
print(dataset.data)
# Define batch size
batch_size = 2 # kalo batch sizenya < ukuran dataset maka max nya = ukuran dataset

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # random urutannya tergantung shuffle, tapi sizenya = batch size

# Iterate through the DataLoader
for batch_idx, (data, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print("Data:", data)  # Should be (batch_size, 10)
    print("Labels:", labels)  # Should be (batch_size,)
    print()


tensor([[-0.0355,  0.9883, -0.1070, -2.2002, -0.1857, -1.2356,  2.4281,  0.7944,
          1.2722, -0.9647],
        [-0.4534,  0.0648,  1.3604, -0.1652,  0.6964,  0.9886,  1.4153, -0.7460,
         -0.3220,  1.1403],
        [ 0.7023, -1.7855,  0.9045,  0.2002,  0.3200, -0.2336, -1.3942, -0.8264,
          0.3473, -1.0203],
        [-1.3067, -1.9278,  0.6478,  0.0377, -1.1756,  0.4332, -0.3931, -1.6890,
         -0.8756, -2.0147],
        [ 0.1667, -0.9539, -0.6144, -0.6683, -0.1462,  0.7636,  0.9793,  0.1480,
         -0.7633, -0.0340],
        [ 0.8234, -0.2369, -0.9035,  0.0839,  0.1560,  1.4537,  1.1638,  0.7818,
          0.9495,  0.2554],
        [-0.3244,  0.5698,  0.3728, -1.2148, -1.1475, -1.2793,  0.1719, -1.8684,
         -0.1628,  0.3270],
        [-0.4933, -0.0396,  1.0276,  1.4835,  0.4210,  0.2828,  0.4031, -0.8791,
          0.2722,  0.4892],
        [-0.4998, -0.9562, -0.1782, -0.4055,  0.7530, -1.2027,  0.2769,  1.6437,
         -1.6612,  0.9675],
        [ 1.3517,  

In [43]:
documents = ["saya pergi ke depok",
             "di depok makan sayuran",
             "dan buah nangka yang segar",
             "angin bertiup kencang",
             "tanda hujan akan turun di jalan margonda"]

dataset = Dataset(2, documents)
dataloader = DataLoader(dataset, batch_size=2) #batch size ganti lagi jadi 2
# print(dataloader)
for batch, (xs, ys) in enumerate(dataloader):
  print(batch)
  print(xs)
  print(ys)

print(dataset.word_to_index)

0
tensor([[2, 3],
        [3, 4]])
tensor([[3, 4],
        [4, 0]])
1
tensor([[4, 0],
        [0, 1]])
tensor([[0, 1],
        [1, 0]])
2
tensor([[1, 0],
        [0, 5]])
tensor([[0, 5],
        [5, 6]])
3
tensor([[5, 6],
        [6, 7]])
tensor([[6, 7],
        [7, 8]])
4
tensor([[7, 8],
        [8, 9]])
tensor([[ 8,  9],
        [ 9, 10]])
5
tensor([[ 9, 10],
        [10, 11]])
tensor([[10, 11],
        [11, 12]])
6
tensor([[11, 12],
        [12, 13]])
tensor([[12, 13],
        [13, 14]])
7
tensor([[13, 14],
        [14, 15]])
tensor([[14, 15],
        [15, 16]])
8
tensor([[15, 16],
        [16, 17]])
tensor([[16, 17],
        [17, 18]])
9
tensor([[17, 18],
        [18,  1]])
tensor([[18,  1],
        [ 1, 19]])
10
tensor([[ 1, 19],
        [19, 20]])
tensor([[19, 20],
        [20, 21]])
{'depok': 0, 'di': 1, 'saya': 2, 'pergi': 3, 'ke': 4, 'makan': 5, 'sayuran': 6, 'dan': 7, 'buah': 8, 'nangka': 9, 'yang': 10, 'segar': 11, 'angin': 12, 'bertiup': 13, 'kencang': 14, 'tanda': 15, 'huj

In [58]:
dataset = Dataset(2, documents)
model = Model(dataset)
# print(model.embedding)
# print(model.rnn)

# training batchnya ada 11 dari 0 sampai 10
train(dataset, model, 2)

ini embedding tensor([[[-0.2300, -1.6504,  0.5657,  0.2887, -0.0433, -0.9523,  0.6938,
           0.6991, -1.4728,  2.4243, -0.3203,  1.7446,  0.6635, -0.0753,
           0.6515, -0.1750],
         [-1.4527,  0.2180,  1.9290, -2.3483, -1.7157, -0.4030, -0.2907,
          -0.5206,  1.0669, -0.2483,  1.6862, -0.2279,  0.6568,  2.0508,
           1.5472, -0.1495]],

        [[-1.4527,  0.2180,  1.9290, -2.3483, -1.7157, -0.4030, -0.2907,
          -0.5206,  1.0669, -0.2483,  1.6862, -0.2279,  0.6568,  2.0508,
           1.5472, -0.1495],
         [-2.0719, -0.1130,  2.0528,  0.6673, -1.5546, -0.6262, -0.0907,
          -1.8288,  0.8053, -0.9332, -0.9966, -1.0704,  1.5957, -1.2276,
          -0.0407, -0.4827]]], grad_fn=<EmbeddingBackward0>). shapenya torch.Size([2, 2, 16])
all inputs tensor([[[-0.2300, -1.6504,  0.5657,  0.2887, -0.0433, -0.9523,  0.6938,
           0.6991, -1.4728,  2.4243, -0.3203,  1.7446,  0.6635, -0.0753,
           0.6515, -0.1750],
         [-1.4527,  0.2180,  1.92

In [ ]:
predict(dataset, model, "saya pergi", next_words=20)

['saya',
 'pergi',
 'ke',
 'depok',
 'di',
 'depok',
 'makan',
 'sayuran',
 'dan',
 'buah',
 'nangka',
 'yang',
 'segar',
 'angin',
 'bertiup',
 'kencang',
 'tanda',
 'hujan',
 'akan',
 'turun',
 'di',
 'depok']